In [ ]:
import sys
import numpy as np
from math import *
import time
from PySide2.QtCore import *
from PySide2.QtGui import *
from PySide2.QtWidgets import *
from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor
from Prog3_ui import Ui_MainWindow
from Prog3_img import Ui_Form
        
if not QApplication.instance():
    app = QApplication(sys.argv)
else:
    app = QApplication.instance()
# app = QApplication(sys.argv)

Form = QMainWindow()
ui = Ui_MainWindow()
ui.setupUi(Form)
Form.show()

def init_const():
    global x_0, y_0, form_x, form_y, d, N
    
    x_0, y_0 = -2, 2
    form_x = ui.lineEdit.text()
    form_y = ui.lineEdit_2.text()
    d = ui.doubleSpinBox.value()
    N = int(4 / d)

def formula(x, y):
    return(eval(form_x), eval(form_y))

def get_num(x, y):
    y = np.where(x < -2, 3, y)
    x = np.where(x < -2, -3, x)
    
    y = np.where(x > 2, 3, y)
    x = np.where(x > 2, -3, x)
    
    x = np.where(y < -2, -3, x)
    y = np.where(y < -2, 3, y)
    
    x = np.where(y > 2, -3, x)
    y = np.where(y > 2, 3, y)
    
    i = ((y_0 - y) // d * N) + ((x - x_0) // d) + 1
    i = np.where(i < 0, 0, i)
    return i[np.where(i > 0)]

def new_num(old_num):
    return 4 * N * ((old_num - 1) // N) + 2 * ((old_num - 1) % N) + 1

def make_graph(graph_keys=None, save=False):
    global x_0, y_0, d, N, form_x, form_y, graph
    if graph_keys == None:
        init_const()
        graph = dict.fromkeys(range(1, N * N + 1))
    else:
        graph_verts = []
#         print(len(graph_keys))
        dots = new_num(np.array(graph_keys))
        N = N * 2
        for dot in dots:
            graph_verts += [dot, dot + 1, dot + N, dot + N + 1]
        new_graph = dict.fromkeys(graph_verts)
        graph = new_graph
        d = d / 2
        ui.doubleSpinBox.setValue(d)
        
    pattern_x = np.ones((10, 10)) * np.array(range(1, 11)) * 0.1 * d - d / 20
    pattern_y = pattern_x.copy().transpose()
    
#     e_cntr = 0
    
    for n in graph.keys():
        y_n = round(y_0 - ((n - 1) // N) * d, 4)
        x_n = round(x_0 + ((n - 1) % N) * d, 4)
        ss = np.unique(get_num(*formula(x_n + pattern_x, y_n - pattern_y)))
        ss = np.array([sss for sss in ss if sss in graph.keys()])
        s = set(ss.astype('int32'))
        graph[n] = s
#         e_cntr += len(s)
            
#     if save:
#         file = 'graph_' + str(N*N) + '.txt'
#         save_graph(file, graph)
    return graph
    
# def save_graph(file, graph):
#     f = open(file, 'w')
#     for k, v in graph.items():
#         f.write(f'{k}: {v}\n')
#     f.close()
    
def invert_graph(graph):
    i_graph = dict.fromkeys(graph.keys(), 0)
    for k, v in graph.items():
        for vertice in v:
            if i_graph[vertice]:
                i_graph[vertice].add(int(k))
            else:
                i_graph[vertice] = {int(k)}
    for k, v in i_graph.items():
        if v == 0:
            i_graph[k] = set()
    return i_graph

def dfs1(v, comp):
    visited[v] = True
    end = True
    for e in graph[v]:
        if not visited[e]:
            end = False
            dfs1(e, comp)
    comp.append(v)
    
def dfs2(v, comp):
    visited[v] = True
    end = True
    for e in i_graph[v]:
        if not visited[e]:
            end = False
            dfs2(e, comp)
    comp.append(v)

def kosaraju(graph):
    global visited
    global i_graph
    i_graph = invert_graph(graph)
    n = N * N
    
#     visited = [False] * (n + 1)
    visited = dict.fromkeys(graph.keys(), False)
    coms = []
    
    for i in range(1, n + 1):
        if (i in graph.keys() and (not visited[i])):
            comp = []
            dfs1(i, comp)
            coms.append(comp)

#     visited = [False] * (n + 1)
    visited = dict.fromkeys(graph.keys(), False)
    c = []

    for comp in coms[::-1]:
        for v in comp[::-1]:
            if not visited[v]:
                co = []
                root = True
                dfs2(v, co)
#                 if len(co) > 1:
                c.append(co)                    
    return c

def scale(x, y):
    return (x + 2) * width // 4 , height - (y + 2) * height // 4

def fill_square(number, d, color):
    y, x = divmod(number - 1, N)
    x, y = x_0 + x * d, y_0 - y * d
    draw.rectangle((*scale(x, y), *scale(x + d, y - d)), fill=color)
    
def draw_grid():
    x, y = 0, 0
    if d > 0.01:
        while x < width:
            x += width / N
            draw.line((x, 0, x, height), fill=ImageColor.getrgb("blue"))
        while y < height:
            y += height / N
            draw.line((0, x, width, x), fill=ImageColor.getrgb("blue"))
            
def draw_axis():
    zero_x = width / 2
    zero_y = height / 2

    draw.line((0, zero_x + 1, height, zero_x + 1))
    draw.line((zero_y + 1, 0, zero_y + 1, width))

def draw_scc(scc, grid=False, axis=False, save=True):
    global height, width, draw, cntr, cntr_edges
    height, width = 800, 800
    image = Image.new("RGB", (width, height))
    draw = ImageDraw.Draw(image)
    draw.rectangle((0, 0, width, height), fill='white')
#     k = 0
    cntr = 0
    cntr_edges = 0
#     for v in scc[np.argsort([len(comp) for comp in scc])[-1]]:
    colors = [(255, 0, 0), (255, 120, 0), (255, 255, 0), (0, 255, 0), (0, 255, 255), (0, 0, 255),
              (120, 0, 255), (255, 0, 255)]
#     colors = colors * divmod(len(scc), len(colors))[0] + colors[:divmod(len(scc), len(colors))[1]]
#     for color, c in zip(colors, scc):
#     i = 0
    for c in scc:
        if len(c) > 1:
#             i += 1
            cntr += len(c)
            for v in c:
                cntr_edges += len(graph[v])
                fill_square(v, d, 'green')
#         else:
#             fill_square(c[0], d, 'pink')
#                 fill_square(v, d, colors[divmod(i, len(colors))[1]])
#     print(f'{cntr} вершин, {cntr_edges} рёбер')

    if axis:
        draw_axis()

    if grid:
        draw_grid()

    if save:
        im_name = 'test2_image_' + str(N * N) + '.png'
        image.save(im_name, "PNG")

def func():
    global graph, scc, was
    start = time.time()
    if not was:
        was = True
        graph = make_graph()
        scc = kosaraju(graph)
        g_time = time.time() - start
        draw_scc(scc)
    for i in range(ui.spinBox.value()):
#         graph_keys = scc[np.argsort([len(comp) for comp in scc])[-1]]
        start = time.time()
        graph_keys = []
#         k = 0
#         for sc in scc:
#             if len(sc) > 1:
#                 k += 1
#         if k < 2:
#             for sc in scc:
#                 for c in sc:
#                     if len(graph[c]) > 0:
#                         graph_keys.append(c)
#         else:
        for sc in scc:
            if len(sc) > 1:
                graph_keys += sc
#             if len(sc) > 1:
#                 graph_keys += sc
#         for gk in graph_keys:
#             for c in graph[gk]:
#                 if c not in graph_keys:
#                     graph_keys.append(c)
        graph = make_graph(graph_keys)
        scc = kosaraju(graph)
#         print(f'{i} итерация, {time.time() - start} времени')
        l_time = time.time() - start
        g_time += l_time
        ui.time_it.setText(str(round(l_time, 3)) + ' c')
        ui.time_all.setText(str(round(g_time, 3)) + ' c')
        draw_scc(scc)
        ui.vertices.setText(str(cntr))
        ui.edges.setText(str(cntr_edges))
        
        
    text = ''
    for k, v in graph.items():
        text += f'{k}: {v}\n'
    ui.textEdit.setText(text)
    text_2 = ''
    for c in scc:
        text_2 += f'{c}\n'
    ui.textEdit_2.setText(text_2)
    
    ui.pushButton_2.setEnabled(True)
    
def show_image():
    global Form2
    Form2 = QWidget()
    ui2 = Ui_Form()
    ui2.setupUi(Form2)
    Form2.show()
    
    scene = QGraphicsScene()
    im_name = 'test2_image_' + str(N * N) + '.png'
    scene.addPixmap(QPixmap(im_name))
    ui2.graphicsView.setScene(scene)
    
was = False

ui.pushButton.clicked.connect(func)
ui.pushButton_2.clicked.connect(show_image)

sys.exit(app.exec_())


In [ ]:
32 вершин, 240 рёбер
0 итерация, 0.07698345184326172 времени
73 вершин, 534 рёбер
1 итерация, 0.23857736587524414 времени
126 вершин, 1009 рёбер
2 итерация, 0.6591906547546387 времени
264 вершин, 2194 рёбер
3 итерация, 2.146732807159424 времени
562 вершин, 4735 рёбер
4 итерация, 7.59108304977417 времени
1094 вершин, 9266 рёбер